In [1]:
import numpy as np
import pandas as pd
import xarray as xr

####  Porosity distribution calculations

In [2]:
def porosity(z, z_swi, phi_swi, phi_inf):
    """Calculates porosity on depth z according to depth at SWI (z_swi),
                  porosity at SWI (phi_swi),
              and porosity at 'infinite depth' layer )phi_inf"""
    k_phi = 0.04
    k = -(z-z_swi)/k_phi
    return (phi_inf+(phi_swi-phi_inf)*np.exp(k))

In [3]:
porosity(2.505, 2.5, 0.75, 0.25)

0.691248451292299

In [4]:
z = np.linspace(2.505, 2.595, 10)
phi_inf_array = np.arange(0.05, 0.36, 0.01)
phi_swi_array = np.arange(0.45, 0.86, 0.01)

which combinations of phi_swi and phi_inf match appoximately 0.43:

In [5]:
for phi_swi in phi_swi_array:
    for phi_inf in phi_inf_array:
        if (0.432 < sum(porosity(z, 2.5, phi_swi, phi_inf))/10 < 0.434):
            print(phi_swi, phi_inf)

0.6100000000000001 0.33
0.6300000000000001 0.32000000000000006
0.6800000000000002 0.29000000000000004
0.7000000000000002 0.28
0.7300000000000002 0.26000000000000006
0.7500000000000002 0.25000000000000006
0.8000000000000003 0.22000000000000003
0.8200000000000003 0.21000000000000002


#### Alkalinity SWI fluxes compounds

In [6]:
ds = xr.open_dataset('data/results/5_po75-25_di10e-9/water.nc')
alkflux_df = ds['B_C_Alk   _flux'].to_dataframe()
nh4flux_df = ds['B_NUT_NH4 _flux'].to_dataframe()
no2flux_df = ds['B_NUT_NO2 _flux'].to_dataframe()
no3flux_df = ds['B_NUT_NO3 _flux'].to_dataframe()
po4flux_df = ds['B_NUT_PO4 _flux'].to_dataframe()
so4flux_df = ds['B_S_SO4   _flux'].to_dataframe()
alkflux_bottom = alkflux_df.groupby('z_faces').get_group(2.5)
nh4flux_bottom = nh4flux_df.groupby('z_faces').get_group(2.5)
no2flux_bottom = no2flux_df.groupby('z_faces').get_group(2.5)
no3flux_bottom = no3flux_df.groupby('z_faces').get_group(2.5)
po4flux_bottom = po4flux_df.groupby('z_faces').get_group(2.5)
so4flux_bottom = so4flux_df.groupby('z_faces').get_group(2.5)

In [7]:
year = (('2011-01-01','2011-01-31'), ('2011-02-01','2011-02-28'), ('2011-03-01','2011-03-31'), ('2011-04-01','2011-04-30'), 
        ('2011-05-01','2011-05-31'), ('2011-06-01','2011-06-30'), ('2011-07-01','2011-07-31'), ('2011-08-01','2011-08-31'),
        ('2011-09-01','2011-09-30'), ('2011-10-01','2011-10-31'), ('2011-11-01','2011-11-30'), ('2011-12-01','2011-12-31'))

In [8]:
year_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

In [9]:
alk_year = []
nh4_year = []
no2_year = []
no3_year = []
po4_year = []
so4_year = []
for month in year:
    alk_month = alkflux_bottom.loc[month[0]:month[1]]
    nh4_month = nh4flux_bottom.loc[month[0]:month[1]]
    no2_month = no2flux_bottom.loc[month[0]:month[1]]
    no3_month = no3flux_bottom.loc[month[0]:month[1]]
    po4_month = po4flux_bottom.loc[month[0]:month[1]]
    so4_month = so4flux_bottom.loc[month[0]:month[1]]
    alk_year.append(alk_month['B_C_Alk   _flux'].mean())
    nh4_year.append(nh4_month['B_NUT_NH4 _flux'].mean())
    no2_year.append(no2_month['B_NUT_NO2 _flux'].mean())
    no3_year.append(no3_month['B_NUT_NO3 _flux'].mean())
    po4_year.append(po4_month['B_NUT_PO4 _flux'].mean())
    so4_year.append(so4_month['B_S_SO4   _flux'].mean())

In [10]:
alk = np.array(alk_year)
nh4 = np.array(nh4_year)
no2 = np.array(no2_year)
no3 = np.array(no3_year)
po4 = np.array(po4_year)
so4 = np.array(so4_year)

In [11]:
all_components = nh4-no2-no3-po4-2*so4

In [12]:
po4_quota = -po4/all_components
po4_quota

array([-0.01286554, -0.01438941, -0.01566453, -0.01458898, -0.01114553,
       -0.00949001, -0.00899623, -0.00885663, -0.00889055, -0.00909168,
       -0.00930219, -0.01058985], dtype=float32)

In [13]:
nh4_quota = nh4/all_components
nh4_quota

array([0.278116  , 0.3257347 , 0.3456419 , 0.2856679 , 0.18116581,
       0.14620319, 0.1382881 , 0.13409601, 0.13027906, 0.12835523,
       0.13665505, 0.19144522], dtype=float32)

In [14]:
no2_quota = -no2/all_components
no2_quota

array([ 0.00168417, -0.00392294, -0.01497258, -0.01415979, -0.00299552,
       -0.00047834, -0.00031866,  0.00020852,  0.0015052 ,  0.00293007,
        0.00639088,  0.00710906], dtype=float32)

In [15]:
no3_quota = -no3/all_components
no3_quota

array([0.17159602, 0.2162815 , 0.2166246 , 0.1292983 , 0.0307871 ,
       0.00402352, 0.00092519, 0.00040211, 0.00082934, 0.00190905,
       0.01730502, 0.07807709], dtype=float32)

In [16]:
so4_quota = -so4*2/all_components
so4_quota

array([0.5614694 , 0.47629616, 0.4683706 , 0.6137826 , 0.80218816,
       0.85974157, 0.87010163, 0.87415004, 0.8762769 , 0.87589735,
       0.8489513 , 0.7339584 ], dtype=float32)

In [17]:
nh4_quota+no2_quota+no3_quota+po4_quota+so4_quota

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       0.99999994, 1.        , 1.        , 0.99999994, 1.        ,
       1.        , 0.99999994], dtype=float32)

#### Alkalinity to reduced sulfur compounds fluxes fractions

In [18]:
ds1 = xr.open_dataset('data/results/2_po75-25_di1e-9/sediments.nc')
ds2 = xr.open_dataset('data/results/5_po75-25_di10e-9/sediments.nc')
ds3 = xr.open_dataset('data/results/10_po75-25_di35e-9/sediments.nc')

In [19]:
ds = ds2

alkflux_df = ds['B_C_Alk   _flux'].to_dataframe()
nh4flux_df = ds['B_NUT_NH4 _flux'].to_dataframe()
no2flux_df = ds['B_NUT_NO2 _flux'].to_dataframe()
no3flux_df = ds['B_NUT_NO3 _flux'].to_dataframe()
po4flux_df = ds['B_NUT_PO4 _flux'].to_dataframe()
so4flux_df = ds['B_S_SO4   _flux'].to_dataframe()
h2s__flux_df = ds['B_S_H2S   _flux'].to_dataframe()
s0___flux_df = ds['B_S_S0    _flux'].to_dataframe()
s2o3_flux_df = ds['B_S_S2O3  _flux'].to_dataframe()

alkflux_bottom = alkflux_df.groupby('z_faces').get_group(2.5)
nh4flux_bottom = nh4flux_df.groupby('z_faces').get_group(2.5)
no2flux_bottom = no2flux_df.groupby('z_faces').get_group(2.5)
no3flux_bottom = no3flux_df.groupby('z_faces').get_group(2.5)
po4flux_bottom = po4flux_df.groupby('z_faces').get_group(2.5)
so4flux_bottom = so4flux_df.groupby('z_faces').get_group(2.5)
h2sflux_bottom  = h2s__flux_df.groupby('z_faces').get_group(2.5)
s0flux_bottom   = s0___flux_df.groupby('z_faces').get_group(2.5)
s2o3flux_bottom = s2o3_flux_df.groupby('z_faces').get_group(2.5)

In [20]:
alkflux_df = alkflux_df.reset_index()
alkflux_df.z_faces[38]

2.5199999809265137

In [21]:
year = (('2011-01-01','2011-01-31'), ('2011-02-01','2011-02-28'), ('2011-03-01','2011-03-31'), ('2011-04-01','2011-04-30'), 
        ('2011-05-01','2011-05-31'), ('2011-06-01','2011-06-30'), ('2011-07-01','2011-07-31'), ('2011-08-01','2011-08-31'),
        ('2011-09-01','2011-09-30'), ('2011-10-01','2011-10-31'), ('2011-11-01','2011-11-30'), ('2011-12-01','2011-12-31'))

In [22]:
alk_year = []
nh4_year = []
no2_year = []
no3_year = []
po4_year = []
so4_year = []
h2s_year = []
s0_year = []
s2o3_year = []
for month in year:
    alk_month = alkflux_bottom.loc[month[0]:month[1]]
    nh4_month = nh4flux_bottom.loc[month[0]:month[1]]
    no2_month = no2flux_bottom.loc[month[0]:month[1]]
    no3_month = no3flux_bottom.loc[month[0]:month[1]]
    po4_month = po4flux_bottom.loc[month[0]:month[1]]
    so4_month = so4flux_bottom.loc[month[0]:month[1]]
    h2s_month = h2sflux_bottom.loc[month[0]:month[1]]
    s0_month = s0flux_bottom.loc[month[0]:month[1]]
    s2o3_month = s2o3flux_bottom.loc[month[0]:month[1]]
    alk_year.append(alk_month['B_C_Alk   _flux'].mean())
    nh4_year.append(nh4_month['B_NUT_NH4 _flux'].mean())
    no2_year.append(no2_month['B_NUT_NO2 _flux'].mean())
    no3_year.append(no3_month['B_NUT_NO3 _flux'].mean())
    po4_year.append(po4_month['B_NUT_PO4 _flux'].mean())
    so4_year.append(so4_month['B_S_SO4   _flux'].mean())
    h2s_year.append(h2s_month['B_S_H2S   _flux'].mean())
    s0_year.append(s0_month['B_S_S0    _flux'].mean())
    s2o3_year.append(s2o3_month['B_S_S2O3  _flux'].mean())

In [23]:
alk = np.array(alk_year)
nh4 = np.array(nh4_year)
no2 = np.array(no2_year)
no3 = np.array(no3_year)
po4 = np.array(po4_year)
so4 = np.array(so4_year)
h2s = np.array(h2s_year)
s0 = np.array(s0_year)
s2o3 = np.array(s2o3_year)

In [24]:
s_total = h2s + s0 + 2*s2o3

In [25]:
so4direct_quota = -2*so4

In [26]:
so4direct_quota

array([ 1.595416 ,  1.1878378,  1.1715816,  2.2493372,  6.868154 ,
       14.1970625, 16.871223 , 14.321139 , 10.899201 ,  8.227101 ,
        6.5335956,  3.7610729], dtype=float32)

In [27]:
sulfurquota = s_total*2

In [28]:
sulfurquota

array([ 3.051776 ,  2.0339575,  1.6073511,  2.009626 ,  4.908166 ,
       10.186014 , 13.106385 , 11.823017 ,  9.78255  ,  8.287424 ,
        7.172136 ,  5.3432584], dtype=float32)

In [29]:
sum(so4direct_quota)-sum(sulfurquota)

8.571062088012695

#### Calculation of the average alkalinity values

In [30]:
base_path = 'data/results'

In [31]:
ds1 = xr.open_dataset('{}/1_po75-25_di0e-9/water.nc'.format(base_path))
ds2 = xr.open_dataset('{}/2_po75-25_di1e-9/water.nc'.format(base_path))
ds3 = xr.open_dataset('{}/3_po75-25_di2e-9/water.nc'.format(base_path))
ds4 = xr.open_dataset('{}/4_po75-25_di5e-9/water.nc'.format(base_path))
ds5 = xr.open_dataset('{}/5_po75-25_di10e-9/water.nc'.format(base_path))
ds6 = xr.open_dataset('{}/6_po75-25_di15e-9/water.nc'.format(base_path))
ds7 = xr.open_dataset('{}/7_po75-25_di20e-9/water.nc'.format(base_path))
ds8 = xr.open_dataset('{}/8_po75-25_di25e-9/water.nc'.format(base_path))
ds9 = xr.open_dataset('{}/9_po75-25_di30e-9/water.nc'.format(base_path))
ds10 = xr.open_dataset('{}/10_po75-25_di35e-9/water.nc'.format(base_path))
ds_no_denitr = xr.open_dataset('data/no_denitrification/water.nc'.format(base_path))

In [32]:
ds = ds_no_denitr

In [33]:
alk_df     = ds['B_C_Alk'].to_dataframe()
alk_surface    = alk_df.groupby('z').get_group(0.625)
alk_surface_year = alk_surface.loc['2011-01-01':'2011-12-31']

In [34]:
alk_surface_year = alk_surface_year.reset_index()

In [35]:
alk_surface_year['B_C_Alk'].mean()

2289.5542